Сервисы доставки еды уже давно перестали быть просто курьерами, которые привозят заказ. Индустрия e-grocery стремительно идет к аккумулированию и использованию больших данных, чтобы знать о своих пользователях больше и предоставлять более качественные и персонализированные услуги. Одним из шагов к такой персонализации может быть разработка модели, которая понимает привычки и нужды пользователя, и, к примеру, может угадать, что и когда пользователь захочет заказать в следующий раз.

Такая модель, будучи разработанной, может принести значительную ценность для клиента - сэкономить время при сборке корзины, помочь ничего не забыть в заказе, убрать необходимость планировать закупки и следить за заканчивающимися запасами продуктов.

В данном соревновании участникам предлагается решить задачу предсказания следующего заказа пользователя (безотносительно конкретного момента времени, когда этот заказ произойдет). Заказ пользователя состоит из списка уникальных категорий товаров, вне зависимости от того, сколько продуктов каждой категории он взял.
# Цель: 
Построить модель, которая будет способна предложить пользователю товар, на основе его предыдущих покупок
# Метрика:
F1 score: $ F1 = 2{pr \over p+r}$, where  $p = {tp \over tp+fp} , r = {tp \over tp+fn}$

# Импорт всех необходимых библиотек
NumPy - для линейной алгебры</br>
Pandas - для работы с csv таблицами</br>
sklearn.model_selection.train_test_split - для разбивки нашего датасета на train и valid</br>
sklearn.metrics.f1_score - для нашей сетрики F1</br>
LAML - модель для предикта

In [ ]:
!pip install -U lightautoml

In [2]:
import numpy as np
import pandas as pd 

import time

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

Прочитаем данные 

In [4]:
df = pd.read_csv('sbermarket-internship-competition/train.csv')
submit_df = pd.read_csv('sbermarket-internship-competition/sample_submission.csv')

В качестве тренировочных данных представляется датасет с историей заказов 20000 пользователей вплоть до даты отсечки, которая разделяет тренировочные и тестовые данные по времени.

train.csv:

user_id - уникальный id пользователя
order_completed_at - дата заказа
cart - список уникальных категорий (category_id), из которых состоял заказ
В качестве прогноза необходимо для каждой пары пользователь-категория из примера сабмита вернуть 1, если категория будет присутствовать в следующем заказе пользователя, или 0 в ином случае. Список категорий для каждого пользователя примере сабмита - это все категории, которые он когда-либо заказывал.

sample_submission.csv:

Пример сабмита. В тест входят не все пользователи из тренировочных данных, так как некоторые из них так ничего и не заказали после даты отсечки.

id - идентификатор строки - состоит из user_id и category_id, разделенных точкой с запятой: f'{user_id};{category_id}'. Из-за особенностей проверяющей системы Kaggle InClass, использовать колонки user_id, category_id в качестве индекса отдельно невозможно
target - 1 или 0 - будет ли данная категория присутствовать в следующем заказе пользователя

Создадим training dataset

Необходимые нам столбцы в таблице: id пользователя, категория, счетчик покупок по каждой категории для каждого пользователя, общий счетчик покупок для каждого пользователя, среднее количество каждой категории в покупке клиента, id как в файле для отправки, целевая переменная (последняя известная покупка)

Также нужно убрать тех людей, которые не встречаются в submission

In [23]:
# Table with True/False for each cart
train_df = pd.get_dummies(df, columns = ['cart'], prefix='', prefix_sep='', dtype='bool')
train_df = train_df.groupby(['user_id', 'order_completed_at']).any().reset_index()

In [24]:
# Count all carts and 
train_df['order_number'] = train_df.groupby(['user_id']).cumcount()
train_df = train_df.drop('order_completed_at', axis=1)

In [25]:
# Last order for each user
last_order = train_df.groupby(['user_id'])['order_number'].transform(max) == train_df['order_number']
train = train_df[~last_order].groupby('user_id').sum().reset_index()
valid = train_df[last_order].reset_index(drop=True)

In [26]:
# Counter for each category for each user
train_melt = pd.melt(train, id_vars=['user_id'], var_name='category', value_name='ordered')
valid_melt = pd.melt(valid, id_vars=['user_id'], var_name='category', value_name='target')

In [27]:
# Total counter for each user
Train = train_melt.copy()

order_number = valid[['user_id', 'order_number']].set_index('user_id').squeeze()
Train['orders_total']= Train['user_id'].map(order_number)

In [28]:
# avg of each category
Train['rating'] = Train['ordered'] / Train['orders_total']

In [29]:
# user_id / category like a submission file
Train['id'] = Train['user_id'].astype(str) + ';' + Train['category']

In [30]:
# Do we know about the previous purchase
Train['target'] = valid_melt['target'].astype(int)

In [32]:
# Remove those users who are not in the submission file
Train = Train[Train.id.isin(submit_df.id.unique())].reset_index(drop=True)

In [33]:
# Counter by all user 
total_ordered = Train.groupby('category')['ordered'].sum()
Train['total_ordered'] = Train['category'].map(total_ordered)

In [35]:
# Let check our Train dataset
Train.head()

А теперь создадим Тrain, на котрый надо сдеалть предикт

In [44]:
Test = Train.copy()


Test['orders_total'] += 1 

#add last buy
Test['ordered'] = Test['ordered'] + Test['target']

#recalculate
test_total_ordered = Test.groupby('category')['ordered'].sum()
Test['total_ordered'] = Test['category'].map(test_total_ordered)
Test['rating'] = Test['ordered'] / Test['orders_total']

Test = Test.drop('target', axis=1)

In [45]:
# Check our Testing dataset
Test.head()

Разделим наши данные на Train и Valid (для валидации 20% и перемешаем наши данные)

In [38]:
Train_set, Valid_set = train_test_split(Train, test_size = 0.2,
                                        stratify = None, random_state = 15)

Инициализируем функцию для подсчета F1

In [39]:
def f1 (real, pred, **kwargs):
    return f1_score(real, (pred > 0.5).astype(int), **kwargs)

Постром нашу модель

In [43]:
%%time 

automl = TabularAutoML(task = Task('binary', metric = f1), 
                       cpu_limit = 4,
                       reader_params = {'n_jobs': 4, 'cv': 5, 'random_state': 15},
                       general_params = {'use_algos': [['linear_l2']]},
                      )
train_pred = automl.fit_predict(Train_set, 
                                roles = {'target': 'target', 'drop': ['user_id', 'category', 'id']})
print('Score on Train', "%.5f" % f1(Train_set.target, train_pred.data))

valid_pred = automl.predict(Valid_set)
print('Score on Valid', "%.5f" % f1(Valid_set.target, valid_pred.data))

Поссмотрим на наши предсказания

In [47]:
predictions = automl.predict(Test)
print('Train mean:', "%.5f" % Train.target.mean())
print('Test mean:', "%.5f" % (predictions.data > 0.5).astype(int).mean())

Значения не подходят, значит надо попробовать поменять пороговое значение

In [48]:
th = 0.5
train_mean = Train.target.mean()
test_mean = (predictions.data > th).astype(int).mean()

while test_mean < train_mean:
    th -= 0.005
    test_mean = (predictions.data > th).astype(int).mean()
    
print('Threshold:', "%.4f" % th)
print('Train mean:', "%.5f" % train_mean)
print('New Test mean:', "%.5f" % test_mean)

Как мы видим оптимальное пороговое значение = 0.24 и среднее на Train-е увеличилось.

Можем сделать последнее предсказание и отправлять решение.

In [50]:
Test['target'] = (predictions.data > th).astype(int)
submit_file = pd.merge(submit_df['id'], Test[['id', 'target']], on='id')
submit_file.to_csv('submission.csv', index = False)

Это было сделано для соревнования от СберМаркета https://www.kaggle.com/c/sbermarket-internship-competition </br>
Место в зачете 14 с Score: 0.48679
<img src="leaderboard.png" />